# Ejercicio ETL

## Extract 

In [1]:
# manejo de rutas entre directorios
from pathlib import Path

path_carga = Path(r"C:\Users\dlare\Progra_TI\ejercitando_py\ejercicio_ventas")


In [2]:
import pandas as pd
import sqlite3
import json

# 1. POS (SQLite)
conn = sqlite3.connect(path_carga/'ventas.db')
pedidos = pd.read_sql('SELECT * FROM pedidos', conn)
conn.close()

# 2. CRM (CSV y Excel)
ventas_csv = pd.read_csv(path_carga/'ventas.csv', encoding="utf-8")
clientes = pd.read_excel(path_carga/'datos.xlsx', sheet_name='Clientes')

# 3. Inventario / Productos (JSON)
with open(path_carga/'productos.json', encoding="utf-8") as f:
    productos = pd.DataFrame(json.load(f))


## Transform

In [3]:
# Convertir fechas a formato con horario
pedidos['fecha'] = pd.to_datetime(pedidos['fecha'])

# Normalizar nombres de producto y detalle ventas (join con JSON)
ventas_csv = ventas_csv.merge(productos, left_on='producto', right_on='nombre', how='left')

# Enriquecer con clientes
pedidos = pedidos.merge(clientes, on='id_cliente', how='left')

# Detectar duplicados
pedidos = pedidos.drop_duplicates()


In [4]:
#chequeo
pedidos

,id_pedido,id_cliente,fecha,total,nombre,ciudad
0,1,1,2024-01-15,1225,Ana,Madrid
1,2,2,2024-01-16,25,Carlos,Barcelona
2,3,1,2024-01-17,380,Ana,Madrid
3,4,3,2024-01-17,150,María,Valencia


In [5]:
#chequeo
ventas_csv

,id_venta,producto,precio,id_pedido,id,nombre,categoria
0,1,Laptop,1200,1,101,Laptop,Electrónica
1,2,Mouse,25,1,102,Mouse,Accesorios
2,3,Mouse,25,2,102,Mouse,Accesorios
3,4,Teclado,80,3,103,Teclado,Accesorios
4,5,Monitor,300,3,104,Monitor,Electrónica
5,6,Audífonos,150,4,105,Audífonos,Accesorios


## Load

In [6]:
# Crear tabla final de ventas consolidadas
ventas_consolidadas = ventas_csv.merge(
    pedidos,
    on='id_pedido',
    how='left'
)

# Añadir columnas extra simuladas
ventas_consolidadas['canal_venta'] = ventas_consolidadas['ciudad'].apply(
    lambda x: 'online' if x == 'Barcelona' else 'tienda'
)
ventas_consolidadas['segmento_cliente'] = ventas_consolidadas['ciudad'].map({
    'Madrid': 'Premium',
    'Barcelona': 'Regular',
    'Valencia': 'Nuevo'
})


In [7]:
#chequeo 

ventas_consolidadas

,id_venta,producto,precio,id_pedido,id,nombre_x,categoria,id_cliente,fecha,total,nombre_y,ciudad,canal_venta,segmento_cliente
0,1,Laptop,1200,1,101,Laptop,Electrónica,1,2024-01-15,1225,Ana,Madrid,tienda,Premium
1,2,Mouse,25,1,102,Mouse,Accesorios,1,2024-01-15,1225,Ana,Madrid,tienda,Premium
2,3,Mouse,25,2,102,Mouse,Accesorios,2,2024-01-16,25,Carlos,Barcelona,online,Regular
3,4,Teclado,80,3,103,Teclado,Accesorios,1,2024-01-17,380,Ana,Madrid,tienda,Premium
4,5,Monitor,300,3,104,Monitor,Electrónica,1,2024-01-17,380,Ana,Madrid,tienda,Premium
5,6,Audífonos,150,4,105,Audífonos,Accesorios,3,2024-01-17,150,María,Valencia,tienda,Nuevo


In [8]:
"""
Cell generated by Data Wrangler.
Normalización de columnas, limpieza de datos
"""
def clean_data(ventas_consolidadas):
    # Rename column 'id' to 'id_producto'
    ventas_consolidadas = ventas_consolidadas.rename(columns={'id': 'id_producto'})
    # Rename column 'total' to 'total_pedido'
    ventas_consolidadas = ventas_consolidadas.rename(columns={'total': 'total_pedido'})
    # Rename column 'nombre_y' to 'nombre_cliente'
    ventas_consolidadas = ventas_consolidadas.rename(columns={'nombre_y': 'nombre_cliente'})
    # Drop column: 'nombre_x'
    ventas_consolidadas = ventas_consolidadas.drop(columns=['nombre_x'])
    return ventas_consolidadas

ventas_consolidadas_clean = clean_data(ventas_consolidadas.copy())
ventas_consolidadas_clean.head()

,id_venta,producto,precio,id_pedido,id_producto,categoria,id_cliente,fecha,total_pedido,nombre_cliente,ciudad,canal_venta,segmento_cliente
0,1,Laptop,1200,1,101,Electrónica,1,2024-01-15,1225,Ana,Madrid,tienda,Premium
1,2,Mouse,25,1,102,Accesorios,1,2024-01-15,1225,Ana,Madrid,tienda,Premium
2,3,Mouse,25,2,102,Accesorios,2,2024-01-16,25,Carlos,Barcelona,online,Regular
3,4,Teclado,80,3,103,Accesorios,1,2024-01-17,380,Ana,Madrid,tienda,Premium
4,5,Monitor,300,3,104,Electrónica,1,2024-01-17,380,Ana,Madrid,tienda,Premium
